### Group redirection. Merging dataframe of ICO features ONLY. Merging 1-Hot encoding columns from Mina with success_df.

In [23]:
import pandas as pd
from path import Path

import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
import timeit
import time
import datetime
import copy

import numpy as np
from sklearn import preprocessing
%matplotlib inline

In [24]:
success_df = pd.read_csv(Path(f'../../../../data/cleandata/merges/success_df.csv'))

In [25]:
one_hot_features = pd.read_csv(Path(f"../../../../data/cleandata/encoded.csv"))

In [26]:
success_df.head(2)

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,custom_index,custom_index_scaled,success
0,2,Ethereum,ETH,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96,Switzerland,...,0 days 00:00:00.000000000,0.460570,0.06116,0.02739,0.90151,100,779,144636.93,0.129162,1
1,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,0 days 00:00:00.000000000,0.563773,0.09667,0.04141,0.86189,100,848,17867.36,0.015956,1


In [27]:
success_df.shape

(573, 29)

In [28]:
success_df.columns

Index(['rank_cnc', 'Name', 'Ticker', 'market_cap', 'price', 'volume24',
       'pct1h', 'pct24h', 'pct7d', 'country', 'platform', 'pre_ico_end',
       'pre_ico_start', 'rating', 'status', 'type', 'End', 'Start', 'Duration',
       'pre_Duration', 'compound', 'negative', 'neutral', 'positive',
       'no_of_posts', 'Longevity', 'custom_index', 'custom_index_scaled',
       'success'],
      dtype='object')

In [29]:
success_df.drop(columns = ['rank_cnc', 'Name', 'market_cap', 'price', 'volume24',
       'pct1h', 'pct24h', 'pct7d', 'country', 'platform', 'pre_ico_end',
       'pre_ico_start', 'rating', 'status', 'type', 'End', 'Start', 'Longevity', 'custom_index', 'custom_index_scaled'], inplace = True)

In [35]:
success_df.columns

Index(['Ticker', 'Duration', 'pre_Duration', 'compound', 'negative', 'neutral',
       'positive', 'no_of_posts', 'success'],
      dtype='object')

In [47]:
success_df.head(10)

,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,success
0,ETH,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.460570,0.061160,0.027390,0.901510,100,1
1,BNB,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100,1
2,EOS,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100,0
3,ADA,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93,0
4,HT,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.525717,0.081630,0.033480,0.884940,100,1
5,XTZ,13 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663939,0.021939,0.878857,0.099327,49,0
6,ATOM,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.340496,0.060687,0.023224,0.916194,67,1
7,DASH,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663146,0.102770,0.025100,0.872110,100,1
8,ETC,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.377329,0.079930,0.037850,0.882180,100,1
9,BAT,0 days 23:00:00.000000000,0 days 00:00:00.000000000,0.830295,0.031600,0.837910,0.130530,100,0


In [39]:
success_df.dtypes

Ticker           object
Duration         object
pre_Duration     object
compound        float64
negative        float64
neutral         float64
positive        float64
no_of_posts       int64
success           int64
dtype: object

In [59]:
for i in range(len(success_df)) : 
    success_df.loc[i, 'Duration'] = success_df.loc[i, 'Duration'][:-24]

In [61]:
for i in range(len(success_df)) : 
    success_df.loc[i, 'pre_Duration'] = success_df.loc[i, 'pre_Duration'][:-24]

In [62]:
success_df.head()

,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,success
0,ETH,0,0,0.460570,0.061160,0.027390,0.901510,100,1
1,BNB,20,0,0.563773,0.096670,0.041410,0.861890,100,1
2,EOS,350,0,0.828470,0.032200,0.858550,0.109270,100,0
3,ADA,488,0,0.589474,0.098387,0.032946,0.868753,93,0
4,HT,0,0,0.525717,0.081630,0.033480,0.884940,100,1


In [63]:
one_hot_features.head(2)

,Ticker,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Belarus,country_Belgium,country_Belize,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,ETH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BNB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
one_hot_features.shape

(573, 84)

In [65]:
merged = pd.merge(left=success_df, right=one_hot_features, left_on='Ticker', right_on='Ticker')

In [66]:
merged.head()

,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,success,country_Argentina,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,ETH,0,0,0.460570,0.061160,0.027390,0.901510,100,1,0,...,0,0,0,0,0,0,0,0,0,0
1,BNB,20,0,0.563773,0.096670,0.041410,0.861890,100,1,0,...,0,0,0,0,0,0,0,0,0,0
2,EOS,350,0,0.828470,0.032200,0.858550,0.109270,100,0,0,...,0,0,0,0,0,0,1,0,0,0
3,ADA,488,0,0.589474,0.098387,0.032946,0.868753,93,0,0,...,0,0,0,0,0,0,1,0,0,0
4,HT,0,0,0.525717,0.081630,0.033480,0.884940,100,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
file_path = Path("../../../../data/cleandata/success_df.csv")
merged.to_csv(file_path, index=False)